In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']=''
%run -i imports.py
jax.config.update('jax_platform_name', 'cpu')
%run -i config.py
config.lamp_substep = 4
%run -i model.py
%run -i shax.py

2022-09-18 17:42:07.569245: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
init_batch = [init() for _ in range(config.shac_N)]
init_batch = state_t(
    *(np.stack(p) for p in zip(*init_batch))
)
theta = policy_net.init(jax.random.PRNGKey(0), jax.vmap(observe)(init_batch))
phi = value_net.init(jax.random.PRNGKey(0), jax.vmap(observe)(init_batch))

with open('weights/xtheta-0.1.bin', 'rb') as f:
    theta = flax.serialization.from_bytes(theta, f.read())

with open('weights/xphi-0.1.bin', 'rb') as f:
    phi = flax.serialization.from_bytes(phi, f.read())

mdp = mdp_t(make_init(0.1), observe, step, terminate)

with open('weights/xtheta-0.1.bin', 'rb') as f:
    theta_01 = flax.serialization.from_bytes(theta, f.read())

with open('weights/xphi-0.1.bin', 'rb') as f:
    phi_01 = flax.serialization.from_bytes(phi, f.read())

with open('weights/xtheta-0.5.bin', 'rb') as f:
    theta_10 = flax.serialization.from_bytes(theta, f.read())

with open('weights/xphi-0.5.bin', 'rb') as f:
    phi_10 = flax.serialization.from_bytes(phi, f.read())

In [3]:
@jax.jit
def cost(schedule, s, z):
    def loop(t_, x):
        t = (t_ + z * len(schedule)) / 500
        # t = np.clip(t, 0, 1)
        s, out = x
        
        old_state = s
        a = policy_net.apply(theta, mdp.observe(s)[None, ...])[0]
        v = value_net.apply(phi, mdp.observe(s)[None, ...])[0]
        a = schedule[t_] + a[0] # * a[1] + a[0]
        s = mdp.step(s, a)
        observe_s = observe(s)
        s_01 = old_state._replace(ms=old_state.ms.at[11:].set(config.lamp_m * 0.1))  # light
        s_10 = old_state._replace(ms=old_state.ms.at[11:].set(config.lamp_m * 0.5))  # heavy
        a_01 = policy_net.apply(theta_01, observe(s_01)[None, ...])[0]
        a_10 = policy_net.apply(theta_10, observe(s_10)[None, ...])[0]
        sp_01 = step(s_01, a_01[0])
        sp_10 = step(s_10, a_10[0])
        obs_cost_01 = np.mean((observe(sp_01) - observe_s) ** 2)
        obs_cost_10 = np.mean((observe(sp_10) - observe_s) ** 2)

        # obs_cost_01 = obs_cost_01 + 0.001 * np.mean((a_01[0] - a)) ** 2
        # obs_cost_10 = obs_cost_10 + 0.001 * np.mean((a_10[0] - a)) ** 2
        # out = out - 1.0 * v / 1000
        out = out + 0.5 * np.mean(s.vs ** 2)
        out = out + 20. * (
            -obs_cost_01
            +obs_cost_10
        ) #* (-1)**(z // 16)
        out = out + 10. * (jax.nn.relu(1700 - v) / 1000)
        return s, out

    s, out = jax.lax.fori_loop(0, len(schedule), loop, (s, 0))
    out = out / len(schedule)
    # out = out + 0.1 * np.mean(np.abs(schedule) ** 2)
    return out, s
dcost = jax.value_and_grad(cost, has_aux=True)

@jax.jit
def loop(s, schedule_t):
    a = policy_net.apply(theta, mdp.observe(s)[None, ...])[0]
    a = schedule_t + a[0] #* a[1] + a[0]
    s = mdp.step(s, a)
    v = value_net.apply(phi, mdp.observe(s)[None, ...])[0]
    return s, v

In [4]:
def make_story(seed):
    prefix = np.zeros((0, 6))

    key = jax.random.PRNGKey(seed)
    s = mdp.init()
    for z in tqdm(range(16)):
        key, subkey = jax.random.split(key)
        # schedule = np.zeros((100, 6))
        schedule = jax.random.normal(subkey, shape=(100, 6)) * 0.01

        opt = optax.sgd(1.0)
        opt_state = opt.init(schedule)

        cs = []
        for t in range(1000 + 1):
            (c, s_), dc = dcost(schedule, s, z)
            # if t % 100 == 0:
            #     print(c)
            cs.append(c)
            opt_updates, opt_state = opt.update(dc, opt_state)
            schedule = optax.apply_updates(schedule, opt_updates)
        (c, s_), dc = dcost(schedule, s, z)
        # plt.figure()
        # plt.plot(cs)
        # plt.show()
        prefix = np.concatenate([prefix, schedule])
        s = s_

        s = mdp.init()
        for t in range(len(prefix)):
            s, v = loop(s, prefix[t])
    
    # import scipy.ndimage
    # plt.figure()
    # plt.plot(vs, 'g+')
    # plt.plot(scipy.ndimage.gaussian_filter(vs, 40), 'k-')
    # plt.xlabel('Time (frame)')
    # plt.ylabel('Value function')
    # plt.axvspan(1.4 * 500, 1.6 * 500, facecolor='r', alpha=0.2)
    # plt.plot(schedule)
    # plt.show()
    
    return prefix

In [5]:
def schedule_to_video(schedule, seed):
    os.system('rm out/*.png')
    import random
    boxcolor = [random.random() for _ in range(3)] + [1]

    vs = []
    s = mdp.init()
    for t in tqdm(range(len(schedule))):
        if t % 2 == 0:
            draw(
                jax.device_get(s),
                f'out/{t:09}',
                boxcolor=boxcolor
                # msg='Light' if ((t // 1600) % 2 == 0) else 'Light'
            )
        s, v = loop(s, schedule[t])
        vs.append(v)

    os.system(f'''../../ffmpeg-git-20220722-i686-static/ffmpeg -hide_banner -loglevel error -framerate 60 -y -pattern_type glob -i 'out/*.png' -c:v libx264 -pix_fmt yuv420p vidout-mime/mime-weight-{seed}-light.mp4''')
    display(Video(f'vidout-mime/mime-weight-{seed}-light.mp4', html_attributes='controls loop autoplay'))
    ns.notify(f'finished mime weight - {seed}')

In [6]:
for seed in range(10):
    schedule = make_story(seed=seed)
    schedule_to_video(schedule, seed)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

In [7]:
# for seed in range(10):
#     display(make_video(
#         mdp_t(make_init(0.1), observe, step, terminate), theta_01, phi_01,
#         seed=seed, name=f'true-weight-{seed}-light', frames=1600, skip=2
#     ))

# for seed in range(10):
#     display(make_video(
#         mdp_t(make_init(0.5), observe, step, terminate), theta_10, phi_10,
#         seed=seed, name=f'true-weight-{seed}-heavy', frames=1600, skip=2
#     ))